In [ ]:
// RK4 tableau
const mjtNum RK4_A[9] = {
  0.5,    0,      0,
  0,      0.5,    0,
  0,      0,      1
};

const mjtNum RK4_B[4] = {
  1.0/6.0, 1.0/3.0, 1.0/3.0, 1.0/6.0
};


// Runge Kutta explicit order-N integrator
//  (A,B) is the tableau, C is set to row_sum(A)
void mj_RungeKutta(const mjModel* m, mjData* d, int N) {
  int nv = m->nv, nq = m->nq, na = m->na;
  mjtNum h = m->opt.timestep, time = d->time;
  mjtNum C[9], T[9], *X[10], *F[10], *dX;
  const mjtNum* A = (N == 4 ? RK4_A : 0);
  const mjtNum* B = (N == 4 ? RK4_B : 0);

  // check order
  if (!A) {
    mjERROR("supported RK orders: N=4");
  }

  // allocate space for intermediate solutions
  mj_markStack(d);
  dX = mjSTACKALLOC(d, 2*nv+na, mjtNum);
  for (int i=0; i < N; i++) {
    X[i] = mjSTACKALLOC(d, nq+nv+na, mjtNum);
    F[i] = mjSTACKALLOC(d, nv+na, mjtNum);
  }

  // precompute C and T;  C,T,A have size (N-1)
  for (int i=1; i < N; i++) {
    // C(i) = sum_j A(i,j)
    C[i-1] = 0;
    for (int j=0; j < i; j++) {
      C[i-1] += A[(i-1)*(N-1)+j];
    }

    // compute T
    T[i-1] = d->time + C[i-1]*h;
  }

  // init X[0], F[0]; mj_forward() was already called
  mju_copy(X[0], d->qpos, nq);
  mju_copy(X[0]+nq, d->qvel, nv);
  mju_copy(F[0], d->qacc, nv);
  if (na) {
    mju_copy(X[0]+nq+nv, d->act, na);
    mju_copy(F[0]+nv, d->act_dot, na);
  }

  // compute the remaining X[i], F[i]
  for (int i=1; i < N; i++) {
    // compute dX
    mju_zero(dX, 2*nv+na);
    for (int j=0; j < i; j++) {
      mju_addToScl(dX, X[j]+nq, A[(i-1)*(N-1)+j], nv);
      mju_addToScl(dX+nv, F[j], A[(i-1)*(N-1)+j], nv+na);
    }

    // compute X[i] = X[0] '+' dX
    mju_copy(X[i], X[0], nq+nv+na);
    mj_integratePos(m, X[i], dX, h);
    mju_addToScl(X[i]+nq, dX+nv, h, nv+na);

    // set X[i], T[i-1] in mjData
    mju_copy(d->qpos, X[i], nq);
    mju_copy(d->qvel, X[i]+nq, nv);
    if (na) {
      mju_copy(d->act, X[i]+nq+nv, na);
    }
    d->time = T[i-1];

    // evaluate F[i]
    mj_forwardSkip(m, d, mjSTAGE_NONE, 1);  // 1: do not recompute sensors and energy
    mju_copy(F[i], d->qacc, nv);
    if (na) {
      mju_copy(F[i]+nv, d->act_dot, na);
    }
  }

  // compute dX for final update (using B instead of A)
  mju_zero(dX, 2*nv+na);
  for (int j=0; j < N; j++) {
    mju_addToScl(dX, X[j]+nq, B[j], nv);
    mju_addToScl(dX+nv, F[j], B[j], nv+na);
  }

  // reset state and time
  d->time = time;
  mju_copy(d->qpos, X[0], nq);
  mju_copy(d->qvel, X[0]+nq, nv);
  mju_copy(d->act, X[0]+nq+nv, na);

  // advance state and time
  mj_advance(m, d, dX+2*nv, dX+nv, dX);

  mj_freeStack(d);
}

All integrators are called from `mj_step()` which selects the appropriate integrator.

The `mj_RungeKutta()` function implements a classical 4th-order Runge-Kutta method. The function begins by defining the RK4 coefficients and allocating space for intermediate calculations. RK4 computes 4 approximations for the derivative at different points: at the beginning of the timestep, two midpoints, and the end of the timestep. The final update is a weighted average of these four slopes/steps using the weights defined in RK4_B.

For each stage, the function calculates the intermediate state by advancing the initial state. For each of the 4 points it sets the next state in `mjData` and calls `mj_forwardSkip()` (this function allows to skip certain computational steps and sensor calculations) to evaluate the system at this state and record the resulting derivatives. Finally, in the last loop it computes the weighted sum of these derivatives and stores them in the `dX` state array. Then it resets the state to the initial values from the current invocation, and lastly calls `mj_advance()` with these weighted accelerations and velocities to advance the state of the simulation.

In [ ]:
// fully implicit in velocity, possibly skipping factorization
void mj_implicitSkip(const mjModel* m, mjData* d, int skipfactor) {
  TM_START;
  int nv = m->nv, nM = m->nM, nD = m->nD;

  mj_markStack(d);
  mjtNum* qfrc = mjSTACKALLOC(d, nv, mjtNum);
  mjtNum* qacc = mjSTACKALLOC(d, nv, mjtNum);

  // set qfrc = qfrc_smooth + qfrc_constraint
  mju_add(qfrc, d->qfrc_smooth, d->qfrc_constraint, nv);

  // IMPLICIT
  if (m->opt.integrator == mjINT_IMPLICIT) {
    if (!skipfactor) {
      // compute analytical derivative qDeriv
      mjd_smooth_vel(m, d, /* flg_bias = */ 1);

      // set qLU = qM
      for (int i=0; i < nD; i++) {
        d->qLU[i] = d->qM[d->mapM2D[i]];
      }

      // set qLU = qM - dt*qDeriv
      mju_addToScl(d->qLU, d->qDeriv, -m->opt.timestep, m->nD);

      // factorize qLU
      int* scratch = mjSTACKALLOC(d, nv, int);
      mju_factorLUSparse(d->qLU, nv, scratch, d->D_rownnz, d->D_rowadr, d->D_colind);
    }

    // solve for qacc: (qM - dt*qDeriv) * qacc = qfrc
    mju_solveLUSparse(qacc, d->qLU, qfrc, nv, d->D_rownnz, d->D_rowadr, d->D_diag, d->D_colind);
  }

  // IMPLICITFAST
  else if (m->opt.integrator == mjINT_IMPLICITFAST) {
    if (!skipfactor) {
      // compute analytical derivative qDeriv; skip rne derivative
      mjd_smooth_vel(m, d, /* flg_bias = */ 0);

      // modified mass matrix MhB = qDeriv[Lower]
      mjtNum* MhB = mjSTACKALLOC(d, nM, mjtNum);
      for (int i=0; i < nM; i++) {
        MhB[i] = d->qDeriv[d->mapD2M[i]];
      }

      // set MhB = M - dt*qDeriv
      mju_addScl(MhB, d->qM, MhB, -m->opt.timestep, nM);

      // copy into qH
      for (int i=0; i < nM; i++) {
        d->qH[i] = MhB[d->mapM2M[i]];
      }

      // factorize in-place
      mj_factorI(d->qH, d->qHDiagInv, nv, d->M_rownnz, d->M_rowadr, m->dof_simplenum, d->M_colind);
    }

    // solve for qacc: (qM - dt*qDeriv) * qacc = qfrc
    mju_copy(qacc, qfrc, nv);
    mj_solveLD(qacc, d->qH, d->qHDiagInv, nv, 1,
               d->M_rownnz, d->M_rowadr, m->dof_simplenum, d->M_colind);

  } else {
    mjERROR("integrator must be implicit or implicitfast");
  }

  // advance state and time
  mj_advance(m, d, d->act_dot, qacc, NULL);

  mj_freeStack(d);

  TM_END(mjTIMER_ADVANCE);
}

In [ ]:
  // computed by mj_implicit/mj_derivative
  mjtNum* qDeriv;            // d (passive + actuator - bias) / d qvel           (nD x 1)

  // computed by mj_implicit/mju_factorLUSparse
  mjtNum* qLU;               // sparse LU of (qM - dt*qDeriv)                    (nD x 1)

What is implicit?

When we talk about numerical integration for physics simulations, we're trying to advance the state of a dynamic system from the current time step to the next. Explicit methods, like Euler or RK4, use information that's already known (current position, velocity, and forces), calculate accelerations at the current state and use those accelerations to directly predict where the system will be in the future.

In implicit methods, we set up an equation where the future state is the unknown. This equation includes how the forces at the future state affect the motion. We solve this equation to find the state that makes the dynamics consistent. Explicit methods are like predicting where a ball will be after throwing it, and implicit methods are like figuring out where you need to catch the ball so that all the physics equations are satisfied.

Looking at the implicit integrator, first the code computes qfrc as the sum of smooth (net unconstrained) forces and constraint forces from mData. Then, it calls `mjd_smooth_vel()` that calculates the partial derivative of the dynamics with respect to velocity, which captures how small changes in velocity affect the forces and resulting accelerations in the system. 

The implicit integrator then sets up the implicit system matrix that will be used to solve for the acceleration with a call to `mju_factorLUSparse()`. This matrix represents how the future state depends on both the current state and the unknown future accelerations. Then, the call to mju_solveLUSparse() solves the matrix. The implicit integrator uses LU factorization which is a decomposition technique to break down a matrix into the product of a lower triangular (L) and an upper triangular (U) matrix. MuJoCo optimizes this by only storing and operating on non-zero elements which helps with the performance when dealing with large, sparse matrices in physics simulations.

Compared to the implicit integrator, the implicitfast integrator calls `mjd_smooth_vel()` with `flag_bias=0` (which subtracts `d qfrc_bias / d qvel from qDeriv` which ignores bias forces). Unlike the implicit, implicitfast uses a special optimized factorization and a specialized solver to advantage of MuJoCo's matrix structure. The implicitfast method uses a simpler derivative approximation and optimized factorization and solver implementations which makes it a good default choice for simulations that benefit from implicit integration (stiff systems, fast-changing forces).

Finally, in both implicit and implicitfast cases the function calls `mj_advance()` to advance the simulation state.

RK4 has higher accuracy than the Euler methods for the same step size, but it's computationally expensive (requires 4 evaluations of the dynamics per step), it's not well suited for stiff systems (which may require very small timesteps), and can become unstable with very small time steps.

RK4: Use when accuracy is important and the system isn't particularly stiff.

Implicit: use for very stiff systems where stability issues arise with other methods.

ImplicitFast: provides the best balance between stability and performance.